In [1]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [2]:
df_predict = pd.read_csv("..\\..\\Documents\\Arcellor_Mittal_project\\CoilData_test.csv")
df_predict.head()

,Unnamed: 0,coil,furnace Number,analyse,Hardness_1,Hardness_2,Width,Temperature before finishing mill,Temperature after finishing mill,Thickness,...,ma,b,n,ti,cr,va,mo,Constriction,Max separation,Number of separation points
0,0,396378,1,K371,10003,101,1302.1,1147,921,4.36,...,291,1,34,6,302,0,25,False,NaN,NaN
1,1,396376,3,K371,10123,101,1282.3,1150,920,4.37,...,384,1,33,12,189,25,7,False,NaN,NaN
2,2,396377,4,K321,10040,102,1297.4,1183,933,4.43,...,463,1,20,11,288,0,40,False,NaN,NaN
3,3,396379,3,K371,10243,102,1295.2,1165,910,4.44,...,296,1,21,9,253,0,9,False,NaN,NaN
4,4,396380,4,K321,10012,100,1293.3,1192,909,3.95,...,329,1,28,8,297,0,23,False,NaN,NaN


In [3]:
df_predict.drop(['analyse','Unnamed: 0', 'coil','Max separation', 'Thickness profile','Hardness_2', 'Number of separation points','furnace Number','b','nb','va','mo','cr'],inplace = True, axis = 1)

In [4]:
"""for column in df_predict[['Hardness_1', 'Width',
       'Temperature before finishing mill', 'Temperature after finishing mill',
       'c', 'mn', 'si', 'p', 's', 'al',
       'ma', 'cr', 'Thickness']]:
    df_predict[column] = pd.to_numeric(df_predict[column], errors="coerce")"""

'for column in df_predict[[\'Hardness_1\', \'Width\',\n       \'Temperature before finishing mill\', \'Temperature after finishing mill\',\n       \'c\', \'mn\', \'si\', \'p\', \'s\', \'al\',\n       \'ma\', \'cr\', \'Thickness\']]:\n    df_predict[column] = pd.to_numeric(df_predict[column], errors="coerce")'

In [5]:
# df_predict.isnull().sum()

In [6]:
df_predict.dropna(axis = 0, how = 'any', inplace = True)

In [7]:
for i in df_predict:
    if df_predict[i].dtypes == "int64":
        df_predict[i] = df_predict[i].astype(float)

In [8]:
df_predict.shape

(55769, 15)

In [9]:
X = df_predict.drop("Constriction", axis=1)
y = df_predict["Constriction"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [10]:
#scale the features
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [11]:
# Balancing the data
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=0, sampling_strategy=0.8)
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)
print(sorted(Counter(y_resampled).items()))

[(False, 1328), (True, 1063)]


In [12]:
clf = RandomForestClassifier(criterion = 'gini', n_estimators = 300, random_state = 30, max_depth = 7)
clf.fit(X_train, y_train)


from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
confusion_matrix(y_pred=y_pred, y_true=y_test)

pd.crosstab(y_test, y_pred, rownames = ['Actual'], colnames =['Predicted'], margins = True)

Predicted,False,True,All
Actual,,,
False,16255,1,16256
True,472,3,475
All,16727,4,16731


In [13]:
clf.score(X_train, y_train)

0.9732312106152979

In [14]:
clf.score(X_test, y_test)

0.9717291255752795

In [15]:
print('Precision: %.3f' % precision_score(y_test, y_pred))
print('Recall: %.3f' % recall_score(y_test, y_pred))
print('Accuracy: %.3f' % accuracy_score(y_test, y_pred))

Precision: 0.750
Recall: 0.006
Accuracy: 0.972


In [16]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

       False       0.97      1.00      0.99     16256
        True       0.75      0.01      0.01       475

    accuracy                           0.97     16731
   macro avg       0.86      0.50      0.50     16731
weighted avg       0.97      0.97      0.96     16731



In [17]:
# Feature importance

feature_importances = pd.DataFrame(clf.feature_importances_, index =X.columns,  columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
Thickness,0.217915
Width,0.102762
mn,0.100625
c,0.099728
si,0.066919
ti,0.059549
Hardness_1,0.054972
ma,0.045413
Temperature before finishing mill,0.045217
Temperature after finishing mill,0.044923


In [18]:
df_predict

,Hardness_1,Width,Temperature before finishing mill,Temperature after finishing mill,Thickness,c,mn,si,p,s,al,ma,n,ti,Constriction
0,10003.0,1302.1,1147.0,921.0,4.36,355.0,2162.0,49.0,133.0,143.0,304.0,291.0,34.0,6.0,False
1,10123.0,1282.3,1150.0,920.0,4.37,551.0,1985.0,101.0,118.0,90.0,395.0,384.0,33.0,12.0,False
2,10040.0,1297.4,1183.0,933.0,4.43,457.0,1895.0,60.0,108.0,115.0,476.0,463.0,20.0,11.0,False
3,10243.0,1295.2,1165.0,910.0,4.44,697.0,2008.0,69.0,139.0,98.0,306.0,296.0,21.0,9.0,False
4,10012.0,1293.3,1192.0,909.0,3.95,477.0,1936.0,52.0,112.0,121.0,340.0,329.0,28.0,8.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55764,9948.0,1257.3,1183.0,913.0,3.27,425.0,2264.0,58.0,97.0,160.0,389.0,339.0,26.0,7.0,False
55765,9951.0,1256.6,1180.0,922.0,2.68,425.0,2283.0,60.0,93.0,160.0,405.0,379.0,24.0,7.0,False
55766,9885.0,1257.2,1182.0,919.0,2.68,357.0,2076.0,99.0,92.0,159.0,334.0,324.0,26.0,8.0,False
55767,9885.0,1257.1,1194.0,923.0,2.68,357.0,2076.0,99.0,92.0,159.0,334.0,324.0,26.0,8.0,False
